In [1]:
import os 
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import cv2
import imageio
import skimage.transform as skTrans
from nilearn.image import resample_img
import nibabel as nib
from tqdm import tqdm
from PIL import Image
from ipywidgets import *

In [3]:
import fastai; fastai.__version__
from fastai.basics import *
from fastai.vision.all import *
from fastai.data.transforms import *

In [4]:
def read_nii(filepath):
    '''
    Reads .nii file and returns pixel array
    '''
    ct_scan = nib.load(filepath)
    array   = ct_scan.get_fdata()
    #array   = np.array(array)
    array   = np.rot90(np.array(array))
    return(array)

In [5]:
# Preprocess the nii file 
# Source https://docs.fast.ai/medical.imaging

dicom_windows = types.SimpleNamespace(
    brain=(80,40),
    subdural=(254,100),
    stroke=(8,32),
    brain_bone=(2800,600),
    brain_soft=(375,40),
    lungs=(1500,-600),
    mediastinum=(350,50),
    abdomen_soft=(400,50),
    liver=(150,30),
    spine_soft=(250,50),
    spine_bone=(1800,400),
    custom = (200,60)
)

@patch
def windowed(self:Tensor, w, l):
    px = self.clone()
    px_min = l - w//2
    px_max = l + w//2
    px[px<px_min] = px_min
    px[px>px_max] = px_max
    return (px-px_min) / (px_max-px_min)

In [6]:
# Preprocessing functions
# Source https://docs.fast.ai/medical.imaging

class TensorCTScan(TensorImageBW): _show_args = {'cmap':'bone'}

@patch
def freqhist_bins(self:Tensor, n_bins=100):
    "A function to split the range of pixel values into groups, such that each group has around the same number of pixels"
    imsd = self.view(-1).sort()[0]
    t = torch.cat([tensor([0.001]),
                   torch.arange(n_bins).float()/n_bins+(1/2/n_bins),
                   tensor([0.999])])
    t = (len(imsd)*t).long()
    return imsd[t].unique()
    
@patch
def hist_scaled(self:Tensor, brks=None):
    "Scales a tensor using `freqhist_bins` to values between 0 and 1"
    if self.device.type=='cuda': return self.hist_scaled_pt(brks)
    if brks is None: brks = self.freqhist_bins()
    ys = np.linspace(0., 1., len(brks))
    x = self.numpy().flatten()
    x = np.interp(x, brks.numpy(), ys)
    return tensor(x).reshape(self.shape).clamp(0.,1.)
    
    
@patch
def to_nchan(x:Tensor, wins, bins=None):
    res = [x.windowed(*win) for win in wins]
    if not isinstance(bins,int) or bins!=0: res.append(x.hist_scaled(bins).clamp(0,1))
    dim = [0,1][x.dim()==3]
    return TensorCTScan(torch.stack(res, dim=dim))

In [7]:
@patch
def save_jpg(x:(Tensor), path, wins, bins=None, quality=90):
    fn = Path(path).with_suffix('.jpg')
    x = (x.to_nchan(wins, bins)*255).byte()
    im = Image.fromarray(x.permute(1,2,0).numpy(), mode=['RGB','CMYK'][x.shape[0]==4])
    #im=im.convert('BGR')
    #im = im.resize((256,256))
    #im = im.rotate(angle = 270)
    #im= np.array(im)
    #im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR)
    #return im
    im.save(fn, quality=quality)

In [11]:
vol_list = []
mask_list = []
for filename in sorted(os.listdir('new/test_data/vol')):
    vol_list.append(filename)
for filename in sorted(os.listdir('new/test_data/mask')):
    mask_list.append(filename)

df_files = pd.DataFrame(vol_list,columns=['volname'])
df_files['maskname'] = mask_list
#df_files=df_files.sort_values(by=['filename'], ascending=True)
df_files

,volname,maskname
0,volume-107.nii,segmentation-107.nii
1,volume-108.nii,segmentation-108.nii
2,volume-109.nii,segmentation-109.nii
3,volume-110.nii,segmentation-110.nii
4,volume-124.nii,segmentation-124.nii
5,volume-125.nii,segmentation-125.nii
6,volume-126.nii,segmentation-126.nii
7,volume-127.nii,segmentation-127.nii
8,volume-22.nii,segmentation-22.nii
9,volume-23.nii,segmentation-23.nii


In [12]:
rows = len(df_files)
dir_path = 'new/test_data/'
for i in range(0,rows):
  vol_path=os.path.join(dir_path+'vol/'+df_files['volname'][i])
  mask_path=os.path.join(dir_path+'mask/'+df_files['maskname'][i])
  curr_ct = read_nii(vol_path)
  curr_mask = read_nii(mask_path)
  curr_dim = curr_mask.shape[2]
  file_name = str(df_files.loc[i,'volname']).split('.')[0]
  mask_name = str(df_files.loc[i,'maskname']).split('.')[0]
  for curr_slice in range(0,curr_dim):
    data = tensor(curr_ct[...,curr_slice].astype(np.float32))
    img = data.save_jpg(f"new/test_data/test_img/vol/{file_name}_slice_{curr_slice}.jpg", [dicom_windows.liver,dicom_windows.custom])     
    mask = Image.fromarray(curr_mask[...,curr_slice].astype('uint8'), mode="L")
    #mask = mask.resize((256,256))
    #mask = mask.rotate(angle=270)
    mask.save(f"new/test_data/test_img/mask/{mask_name}_slice_{curr_slice}.tiff")
  print('complted',i)

complted 0
complted 1
complted 2
complted 3
complted 4
complted 5
complted 6
complted 7
complted 8
complted 9
complted 10
complted 11
complted 12
complted 13
complted 14
complted 15
complted 16
complted 17
complted 18
complted 19
complted 20
complted 21
complted 22
complted 23
complted 24
complted 25
complted 26
complted 27
complted 28
complted 29
complted 30
complted 31


In [10]:
rows = len(df_files)
dir_path = 'new/train_data/'
for i in range(0,rows):
  vol_path=os.path.join(dir_path+'vol/'+df_files['volname'][i])
  mask_path=os.path.join(dir_path+'mask/'+df_files['maskname'][i])
  curr_ct = read_nii(vol_path)
  curr_mask = read_nii(mask_path)
  curr_dim = curr_mask.shape[2]
  file_name = str(df_files.loc[i,'volname']).split('.')[0]
  mask_name = str(df_files.loc[i,'maskname']).split('.')[0]
  for curr_slice in range(0,curr_dim):
    data = tensor(curr_ct[...,curr_slice].astype(np.float32))   
    mask = curr_mask[...,curr_slice]
    if (1 in np.unique(mask)):
        img = data.save_jpg(f"new/train_data/train_img/liver/{file_name}_slice_{curr_slice}.jpg", [dicom_windows.liver,dicom_windows.custom])
        mask = Image.fromarray(curr_mask[...,curr_slice].astype('uint8'), mode="L")
        #mask = mask.resize((256,256))
        #mask = mask.rotate(angle=270)
        mask.save(f"new/train_data/train_img/mask/{mask_name}_slice_{curr_slice}.tiff")
    #else:
        
        #img = data.save_jpg(f"new/final/Train_img/bg/{file_name}_slice_{curr_slice}.jpg", [dicom_windows.liver,dicom_windows.custom])
  print('complted',i)

complted 0
complted 1
complted 2
complted 3
complted 4
complted 5
complted 6
complted 7
complted 8
complted 9
complted 10
complted 11
complted 12
complted 13
complted 14
complted 15
complted 16
complted 17
complted 18
complted 19
complted 20
complted 21
complted 22
complted 23
complted 24
complted 25
complted 26
complted 27
complted 28
complted 29
complted 30
complted 31
complted 32
complted 33
complted 34
complted 35
complted 36
complted 37
complted 38
complted 39
complted 40
complted 41
complted 42
complted 43
complted 44
complted 45
complted 46
complted 47
complted 48
complted 49
complted 50
complted 51
complted 52
complted 53
complted 54
complted 55
complted 56
complted 57
complted 58
complted 59
complted 60
complted 61
complted 62
complted 63
complted 64
complted 65
complted 66
complted 67
complted 68
complted 69
complted 70
complted 71
complted 72
complted 73
complted 74
complted 75
complted 76
complted 77
complted 78
complted 79
complted 80
complted 81
complted 82
complted 83
co

In [26]:
a=np.array([1,2,3])
if(3 in a):
    print('yes')
else:
    print('no')
        

yes


In [25]:
5 in a

False